# Instalasi Library

In [17]:
!pip install yfinance -q

# Impor Library

In [18]:
import yfinance as yf
import pandas as pd
import numpy as np

# Konfigurasi Global & Nama Kunci (Keys)

In [19]:
# KONFIGURASI NAMA KUNCI LAPORAN KEUANGAN
KEY_NET_INCOME = 'Net Income'
KEY_REVENUE = 'Total Revenue'
KEY_EQUITY = 'Stockholders Equity'
KEY_DEBT = 'Total Debt'

# Fungsi Pengambilan Rasio (PER, PBV)

In [20]:
def get_info_ratios(ticker_obj):
    """
    Mengambil rasio TTM yang sudah jadi (PER & PBV) dari .info.

    Args:
        ticker_obj (yf.Ticker): Objek Ticker yfinance.

    Returns:
        dict: Dictionary berisi PER dan PBV (atau np.nan jika gagal).
    """
    try:
        info = ticker_obj.info
        per = info.get('trailingPE')
        pbv = info.get('priceToBook')
        return {'PER': per, 'PBV': pbv}

    except Exception as e:
        print(f"  > Gagal mengambil data '.info': {e}")
        return {'PER': np.nan, 'PBV': np.nan}

# Fungsi Pengambilan Rasio (NPM, ROE, DER)

In [21]:
def calculate_ttm_ratios(ticker_obj):
    """
    Menghitung rasio NPM, ROE, dan DER berdasarkan data TTM dari
    laporan keuangan kuartalan.

    Args:
        ticker_obj (yf.Ticker): Objek Ticker yfinance.

    Returns:
        dict: Dictionary berisi NPM, ROE, dan DER (atau np.nan jika gagal).
    """
    results = {'NPM': np.nan, 'ROE': np.nan, 'DER': np.nan}

    try:
        # 1. Ambil Laporan Keuangan KUARTALAN
        laba_rugi_q = ticker_obj.quarterly_financials
        neraca_q = ticker_obj.quarterly_balance_sheet

        # 2. Hitung Data Arus (Flow) TTM (Total 4 Kuartal Terakhir)
        laba_bersih_ttm = laba_rugi_q.loc[KEY_NET_INCOME].iloc[0:4].sum()
        pendapatan_ttm = laba_rugi_q.loc[KEY_REVENUE].iloc[0:4].sum()

        # 3. Ambil Data Stok (Stock) Terbaru (1 Kuartal Terakhir)
        ekuitas_q = neraca_q.loc[KEY_EQUITY].iloc[0]

        # 4. Hitung Rasio (dengan proteksi pembagian nol)
        if pendapatan_ttm > 0:
            results['NPM'] = (laba_bersih_ttm / pendapatan_ttm)

        if ekuitas_q > 0:
            results['ROE'] = (laba_bersih_ttm / ekuitas_q)

        # 5. Hitung DER (dipisah karena KEY_DEBT sering error)
        try:
            total_hutang_q = neraca_q.loc[KEY_DEBT].iloc[0]
            if ekuitas_q > 0:
                results['DER'] = (total_hutang_q / ekuitas_q)
        except KeyError:
            print(f"  > Peringatan: Key '{KEY_DEBT}' tidak ditemukan.")

    except KeyError as e:
        print(f"  > GAGAL TTM (KeyError): Kunci data tidak ditemukan: {e}")
    except Exception as e:
        print(f"  > GAGAL TTM (Error Lain): {e}") # Misal data < 4 kuartal

    return results

# Daftar Saham

In [22]:
ticker_list = [
    # FINANCE
    "BBCA", "BBRI", "BMRI", "BBNI", "BRIS", "BBTN", "COIN", "ARTO", "BNGA", "BTPS", "BJTM", "BBKP", "SRTG", "BBYB", "AGRO", "NISP", "BJBR", "PNLF", "BFIN", "BDMN", "BBHI", "BABP", "BNLI", "PNBS", "BANK", "INPC", "BGTG", "BNII", "BCAP", "ADMF", "PNBN", "CFIN", "BTPN", "MEGA", "BACA", "BEKS", "MAYA", "PALM", "NOBU", "BVIC", "TUGU", "BSIM", "BNBA", "LPPS", "AMAR", "MCOR", "PNIN", "PANS", "DNET", "PADI", "TRIM", "AGRS", "DHAR", "AHAP", "SMMA", "BKSW", "WOMF", "JMAS", "BINA", "VTNY", "VICO", "PEGE", "LIFE", "GSMF", "MTWI", "LPGI", "POLA", "BCIC", "BMAS", "SDRA", "FUJI", "BBSI", "VINS", "AMAG", "VRNA", "ASMI", "CASA", "ASDM", "AMOR", "APIC", "HDFA", "RELI", "YOII", "STAR", "NICK", "MREI", "TRUS", "BHAT", "ASRM", "ASJT", "TIFA", "BBLD", "ASBI", "BPFI", "BBMD", "MASB", "BPII", "DEFI", "YULE", "POOL", "SFAN", "ABDA", "BSWD", "OCAP", "PLAS",

    # BASIC-IND
    "ANTM", "SMGR", "BRPT", "INTP", "BRMS", "EMAS", "MDKA", "INCO", "TINS", "TPIA", "INKP", "MBMA", "PSAB", "AMMN", "TKIM", "ESSA", "ARCI", "NCKL", "KRAS", "DKFT", "WSBP", "NICL", "SMBR", "WTON", "SOLA", "AGII", "AVIA", "NIKL", "MINE", "ISSP", "DAAZ", "SMGA", "OKAS", "ZINC", "NICE", "BAJA", "CHEM", "EKAD", "PPRI", "FPNI", "PBID", "AYLS", "OPMS", "SPMA", "PICO", "SRSN", "GDST", "ASPR", "CLPI", "SMLE", "MDKI", "BLES", "SULI", "CITA",
    "BEBS", "BATR", "ADMG", "LTLS", "DGWG", "ESIP", "SAMF", "FWCT", "NPGF", "TBMS", "KAYU", "SBMA", "PTMR", "SMCB", "PDPP", "BMSR", "UNIC", "OBMD", "SQMI", "PURE", "INCI", "TIRT", "SWAT", "INCF", "EPAC", "GGRP", "HKMU", "ALDO", "IFII", "IGAR", "SMKL", "BTON", "IPOL", "MOLI", "IFSH", "KDSI", "INAI", "INRU", "INTD", "CMNT", "ALKA", "KMTR", "DPNS", "AKPI", "CTBN", "TALF", "APLI", "KKES", "BRNA", "TRST", "YPAS", "LMSH", "ALMI", "FASW", "TDPM", "ETWA", "SIMA", "KBRI",

    # CYCLICAL
    "MNCN", "SCMA", "LPPF", "ACES", "ERAA", "MINA", "MAPI", "GJTL", "AUTO", "BUVA", "BMTR", "MPMX", "FUTR", "HRTA", "FILM", "RALS", "KPIG", "MAPA", "MERI", "FAST", "SLIS", "DOOH", "SMSM", "IMAS", "KAQI", "ZATA", "WOOD", "MSIN", "CNMA", "VKTR", "ASLC", "SRIL", "IPTV", "MSKY", "BELL", "PMUI", "JIHD", "TMPO", "CARS", "ABBA", "NETV", "PZZA", "DRMA", "MARI", "DOSS", "ERAL", "ACRO", "VIVA", "ESTA", "PSKT", "INDR", "VERN", "YELO", "JGLE", "MDIY", "EAST", "RAAM", "PART", "GEMA", "LIVE", "GRPH", "PBRX", "KOTA", "PJAA", "PANR", "FORU", "BOLA", "BAIK", "GOLF", "FITT", "LPIN", "INDS", "TRIS", "DFAM", "ERTX", "UFOE", "ECII", "TOOL", "BAYU", "KICI", "MAPB", "MDIA", "SWID", "TOYS", "INOV", "BAUT", "GDYR", "POLU", "DEPO", "ESTI", "UNTD", "SONA", "HRME", "CSMI", "BIMA", "RAFI", "BABY", "DIGI", "POLY", "LMPI", "CINT", "GWSA", "NATO", "TELE", "JSPT", "SHID", "BIKE", "BOGA", "PGLI", "DUCK", "ENAK", "CSAP", "BRAM", "SSTM", "MICE", "MASA", "RICY", "PMJS", "SOTS", "PTSP", "SNLK", "ARGO", "BOLT", "BATA", "PNSE", "ZONE", "SCNP", "TYRE", "CLAY", "SBAT", "IIKP", "KDTN", "ARTA", "CBMF", "HOME", "PDES", "MGNA", "BLTZ", "GLOB", "MKNT", "TFCO", "MYTX", "AKKU", "NUSA", "CNTX", "TRIO", "MABA", "HOTL", "UNIT",

    # ENERGY
    "ADRO", "PGAS", "PTBA", "ITMG", "BUMI", "HRUM", "PTRO", "CUAN", "MEDC", "ADMR", "RAJA", "AADI", "AKRA", "INDY", "ENRG", "ELSA", "TOBA", "DEWA", "RATU", "BSSR", "HUMI", "ATLA", "CBRE", "PSAT", "DOID", "GEMS", "BULL", "BYAN", "BIPI", "ABMM", "DSSA", "KKGI", "TEBE", "TPMA", "SGER", "FIRE", "MBAP", "SOCI", "GTSI", "MBSS", "APEX", "CGAS", "MTFN", "LEAD", "COAL", "WINS", "IATA", "SMMT", "WOWS", "PSSI", "SICO", "BOAT", "MAHA", "UNIQ", "BSML", "ITMA", "BOSS", "CNKO", "TCPI", "RUIS", "RMKE", "JSKY", "MYOH", "TAMU", "MCOL", "RGAS", "DWGL", "HILL", "ARII", "GTBO", "BBRM", "RMKO", "SHIP", "PKPK", "SEMA", "RIGS", "BESS", "ALII", "KOPI", "TRAM", "ARTI", "INPS", "PTIS", "AIMS", "MKAP", "HITS", "SUNI", "SURE", "CANI", "SMRU", "SUGI",

    # HEALTH
    "KLBF", "SIDO", "KAEF", "MIKA", "PYFA", "SILO", "DKHH", "HEAL", "INAF", "TSPC", "CHEK", "IRRA", "SURI", "PRDA", "SAME", "OBAT", "MDLA", "PRIM", "MERK", "PEHA", "HALO", "MEDS", "MMIX", "DGNS", "DVLA", "CARE", "SOHO", "SRAJ", "BMHS", "OMED", "PEVE", "LABS", "MTMH", "RSCH", "SCPI", "IKPM", "PRAY", "RSGK",

    # INDUSTRIAL
    "ASII", "UNTR", "HEXA", "PIPA", "SMIL", "NAIK", "LABA", "TOTO", "MHKI", "BHIT", "PTMP", "MARK", "ASGR", "ARNA", "INDX", "BNBR", "GPSO", "IMPC", "KBLI", "KOBX", "SPTO", "CAKK", "MLIA", "CCSI", "NTBK", "DYAN", "SKRN", "IKAI", "MUTU", "MDRN", "JTPE", "LION", "KUAS", "SCCO", "KBLM", "VOKS", "HOPE", "VISI", "SINI", "BLUE", "JECC", "ARKA", "PADA", "AMIN", "IKBI", "AMFG", "SOSS", "FOLK", "ZBRA", "ICON", "BINO", "KONI", "INTA", "KIAS", "TIRA", "CRSN", "MFMI", "HYGN", "IBFN", "APII", "CTTH", "KOIN", "TRIL",

    # INFRASTRUC
    "TLKM", "ADHI", "CDIA", "JSMR", "WIKA", "PTPP", "WSKT", "BREN", "PGEO", "EXCL", "ISAT", "TOWR", "INET", "SSIA", "DATA", "POWR", "TBIG", "WEGE", "TOTL", "IPCC", "MTEL", "KOKA", "KRYA", "KBLV", "NRCA", "OASA", "PPRE", "ACST", "JKON", "JAST", "CENT", "KEEN", "LINK", "GMFI", "HGII", "IPCM", "PBSA", "CMNP", "MPOW", "KETR", "ARKO", "META", "DGIK", "KARW", "BDKR", "ASLI", "TGRA", "CASS", "BALI", "BUKK", "IDPR", "HADE", "MTPS", "GOLD", "TAMA", "TOPS", "PTDU", "GHON", "MORA", "PORT", "RONY", "SUPR", "BTEL", "IBST", "PTPW", "LCKM", "MTRA",

    # NON-CYCLICAL
    "UNVR", "INDF", "ICBP", "GGRM", "AALI", "JPFA", "MYOR", "CPIN", "HMSP", "LSIP", "AMRT", "MLPL", "ULTJ", "FORE", "GZCO", "TAPG", "CLEO", "BWPT", "SIMP", "WIIM", "MPPA", "HOKI", "JARR", "CPRO", "DSNG", "IKAN", "SMAR", "GOOD", "SSMS", "BTEK", "MIDI", "MAIN", "COCO", "PGUN", "CAMP", "ANJT", "WMUU", "SGRO", "PTPS", "AISA", "TBLA", "CMRY", "ROTI", "ITIC", "NASI", "BEEF", "BRRC", "OILS", "DSFI", "KEJU", "CSRA", "CEKA", "ADES", "KINO", "ISEA", "NSSS", "STRK", "MRAT", "STAA", "PMMP", "ASHA", "BISI", "JAWA", "WAPO", "RANC", "UCID", "MBTO", "GULA", "ANDI", "DEWI", "TAYS", "DMND", "YUPI", "BUDI", "MLBI", "PSDN", "NEST", "FOOD", "AYAM", "STTP", "LAPD", "MKTR", "PSGO", "TLDN", "PCAR", "DLTA", "EPMT", "TGUK", "MAXI", "GUNA", "PNGO", "HERO", "SKBM", "SKLT", "UNSP", "MGRO", "BEER", "SDPC", "TCID", "FISH", "IPPE", "DPUM", "MSJA", "KMDS", "BUAH", "TGKA", "ENZO", "BOBA", "WMPP", "WINE", "CBUT", "VICI", "DAYA", "CRAB", "AGAR", "FAPA", "SIPD", "TRGU", "ALTO", "GOLL", "WICO", "MAGP",

    # PROPERTY
    "CTRA", "BSDE", "PWON", "SMRA", "PANI", "KIJA", "CBDK", "DMAS", "DADA", "ASRI", "BKSL", "LPKR", "BSBK", "APLN", "REAL", "DILD", "BEST", "PPRO", "MMLP", "LPCK", "UANG", "PAMG", "MDLN", "GPRA", "ADCP", "ELTY", "JRPT", "BCIP", "BAPA", "WINR", "SMDM", "CSIS", "KOCI", "TRIN", "KSIX", "CITY", "AMAN", "KBAG", "NZIA", "TRUE", "LAND", "DUTI", "INDO", "ASPI", "GRIA", "BAPI", "LPLI", "SAGE", "RBMS", "TARA", "MTLA", "VAST", "PURI", "SATU", "RODA", "EMDE", "RISE", "MPRO", "HOMI", "PLIN", "BBSS", "FMII", "ATAP", "PUDP", "NASA", "MKPI", "BKDP", "RDTX", "CBPE", "URBN", "POLL", "GMTD", "BIPP", "DART", "MTSM", "INPP", "OMRE", "POSA", "POLI", "ROCK", "NIRO", "BIKA", "CPRI", "RIMO", "ARMY", "GAMA", "LCGP", "COWL",

    # TECHNOLOGY
    "GOTO", "WIFI", "EMTK", "BUKA", "WIRG", "IOTF", "DCII", "MTDL", "DMMX", "MLPT", "ELIT", "TOSK", "KREN", "PTSN", "KIOS", "CYBR", "JATI", "LUCK", "IRSX", "EDGE", "TFAS", "MCAS", "DIVA", "MSTI", "ZYRX", "MPIX", "HDIT", "TRON", "NFCX", "AWAN", "UVCR", "AREA", "AXIO", "BELI", "ATIC", "GLVA", "ENVY", "TECH", "LMAS", "SKYB",

    # TRANSPORT
    "GIAA", "SMDR", "BLOG", "BIRD", "ASSA", "TMAS", "IMJS", "LAJU", "CMPP", "HAIS", "PURA", "NELY", "TNCA", "KJEN", "JAYA", "MITI", "MPXL", "TAXI", "WEHA", "TRUK", "HATM", "GTRA", "SAPX", "AKSI", "MIRA", "SDMU", "ELPI", "TRJA", "KLAS", "BLTA", "BPTR", "SAFE", "HELI", "LRNA", "DEAL",
]

# Pengumpulan Data

In [23]:
# List kosong untuk menampung semua dictionary data
all_stock_data = []

print(f"Memulai proses TTM untuk {len(ticker_list)} saham\n")

# Loop utama untuk setiap ticker
for ticker in ticker_list:
    ticker_symbol = f"{ticker}.JK"
    print(f"Menganalisis Saham: {ticker_symbol}")

    # Buat objek Ticker
    saham_obj = yf.Ticker(ticker_symbol)

    # Siapkan dictionary untuk menampung hasil satu saham ini
    current_stock_data = {'Ticker': ticker}

    # 1. Ambil rasio dari .info (PER, PBV)
    info_data = get_info_ratios(saham_obj)
    current_stock_data.update(info_data)

    # 2. Hitung rasio TTM (NPM, ROE, DER)
    ttm_data = calculate_ttm_ratios(saham_obj)
    current_stock_data.update(ttm_data)

    # 3. Masukkan hasil saham ini ke list utama
    all_stock_data.append(current_stock_data)

print("\nProses Selesai")

Memulai proses TTM untuk 910 saham

Menganalisis Saham: BBCA.JK
Menganalisis Saham: BBRI.JK
Menganalisis Saham: BMRI.JK
Menganalisis Saham: BBNI.JK
Menganalisis Saham: BRIS.JK
Menganalisis Saham: BBTN.JK
Menganalisis Saham: COIN.JK
  > GAGAL TTM (KeyError): Kunci data tidak ditemukan: 'Net Income'
Menganalisis Saham: ARTO.JK
Menganalisis Saham: BNGA.JK
Menganalisis Saham: BTPS.JK
Menganalisis Saham: BJTM.JK
Menganalisis Saham: BBKP.JK
Menganalisis Saham: SRTG.JK
Menganalisis Saham: BBYB.JK
Menganalisis Saham: AGRO.JK
Menganalisis Saham: NISP.JK
Menganalisis Saham: BJBR.JK
Menganalisis Saham: PNLF.JK
Menganalisis Saham: BFIN.JK
Menganalisis Saham: BDMN.JK
Menganalisis Saham: BBHI.JK
Menganalisis Saham: BABP.JK
Menganalisis Saham: BNLI.JK
Menganalisis Saham: PNBS.JK
Menganalisis Saham: BANK.JK
Menganalisis Saham: INPC.JK
Menganalisis Saham: BGTG.JK
Menganalisis Saham: BNII.JK
Menganalisis Saham: BCAP.JK
Menganalisis Saham: ADMF.JK
Menganalisis Saham: PNBN.JK
Menganalisis Saham: CFIN.JK
M

ERROR:yfinance:HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: DHAR.JK"}}}


  > GAGAL TTM (KeyError): Kunci data tidak ditemukan: 'Net Income'
Menganalisis Saham: AHAP.JK
  > GAGAL TTM (KeyError): Kunci data tidak ditemukan: 'Net Income'
Menganalisis Saham: SMMA.JK
Menganalisis Saham: BKSW.JK
Menganalisis Saham: WOMF.JK
Menganalisis Saham: JMAS.JK
  > GAGAL TTM (KeyError): Kunci data tidak ditemukan: 'Net Income'
Menganalisis Saham: BINA.JK
Menganalisis Saham: VTNY.JK
Menganalisis Saham: VICO.JK
Menganalisis Saham: PEGE.JK
  > Peringatan: Key 'Total Debt' tidak ditemukan.
Menganalisis Saham: LIFE.JK
Menganalisis Saham: GSMF.JK
Menganalisis Saham: MTWI.JK
  > GAGAL TTM (KeyError): Kunci data tidak ditemukan: 'Net Income'
Menganalisis Saham: LPGI.JK
Menganalisis Saham: POLA.JK
Menganalisis Saham: BCIC.JK
Menganalisis Saham: BMAS.JK
Menganalisis Saham: SDRA.JK
Menganalisis Saham: FUJI.JK
Menganalisis Saham: BBSI.JK
Menganalisis Saham: VINS.JK
  > GAGAL TTM (KeyError): Kunci data tidak ditemukan: 'Net Income'
Menganalisis Saham: AMAG.JK
  > Peringatan: Key 'Total 

# Pembuatan DataFrame & Pelabelan

In [24]:
# Buat DataFrame dari list berisi dictionary
df = pd.DataFrame(all_stock_data)

# Mengatur kolom agar urutannya sesuai
df = df[['Ticker', 'NPM', 'ROE', 'PER', 'PBV', 'DER']]

# Paksa semua kolom rasio menjadi numerik (angka).
# errors='coerce' akan mengubah semua nilai non-angka (seperti None atau teks)
# menjadi NaN (Not a Number), yang bisa diproses dengan benar.
df['NPM'] = pd.to_numeric(df['NPM'], errors='coerce')
df['ROE'] = pd.to_numeric(df['ROE'], errors='coerce')
df['PER'] = pd.to_numeric(df['PER'], errors='coerce')
df['PBV'] = pd.to_numeric(df['PBV'], errors='coerce')
df['DER'] = pd.to_numeric(df['DER'], errors='coerce')

# Aturan berdasarkan preferensi Romi
cond_npm = df['NPM'] > 0.05
cond_roe = df['ROE'] > 0.10
cond_per = df['PER'] < 15
cond_pbv = df['PBV'] < 1.5
cond_der = df['DER'] < 1

# Gabungkan semua kondisi (semua harus True)
all_conditions = cond_npm & cond_roe & cond_per & cond_pbv & cond_der

# Buat kolom 'Label' baru
df['Label'] = np.where(all_conditions, 'Memenuhi', 'Tidak Memenuhi')

# Catatan: Saham dengan data NaN (hasil 'coerce') akan otomatis
# gagal perbandingan dan mendapat label 'Tidak Memenuhi'.

# Tampilkan Hasil Akhir & Simpan ke File

In [25]:
# Tampilkan DataFrame final dengan label
print("DataFrame Final dengan Label:\n")
print(df)

# Menyimpan ke CSV di lingkungan Colab
csv_filename = "dataset_stock_screener.csv"
df.to_csv(csv_filename, index=False)
print(f"\nData berlabel berhasil disimpan ke '{csv_filename}'")

DataFrame Final dengan Label:

    Ticker       NPM       ROE        PER       PBV       DER           Label
0     BBCA  0.497092  0.217804  18.651888  3.856816  0.006831  Tidak Memenuhi
1     BBRI  0.318951  0.179808  10.634337  1.909198  0.553886  Tidak Memenuhi
2     BMRI  0.392909  0.201205   8.612561  1.574164  0.728915  Tidak Memenuhi
3     BBNI  0.327198  0.130113   8.140539  0.989747  0.409555        Memenuhi
4     BRIS  0.323124  0.153746  15.934778  2.488623  0.139440  Tidak Memenuhi
..     ...       ...       ...        ...       ...       ...             ...
905   BPTR       NaN       NaN  11.108428  0.555043       NaN  Tidak Memenuhi
906   SAFE  0.095192  1.952640   4.878745  9.529117  0.556297  Tidak Memenuhi
907   HELI  0.060966  0.124520  24.897120  3.098711  1.431335  Tidak Memenuhi
908   LRNA       NaN       NaN        NaN  0.221954       NaN  Tidak Memenuhi
909   DEAL       NaN       NaN        NaN -0.120758       NaN  Tidak Memenuhi

[910 rows x 7 columns]

Data ber